# Script Demo

This is an early of UCSD's markdown to lecture pipeline through HeyGen. The way it works is you edit the example .md files and feed the name to the second code cell in this notebook. An example and discussion of the syntax can be found in the $\verb|example.md|$. A list of voice and avatar ids is available in the files $\verb|voices.txt|$ and $\verb|avatars.txt|$.

Please note that HeyGen seems to have caught on to my strategy of generating an arbitrary number of free trial api keys by using Apple's hide my email function. Calls from those APIs made with a jupyter notebook seem to take forever (I mean about 20 minutes) whereas on my laptop they take about 2 minutes. I need a non-trial APi key to keep doing this at scale. Keep that in mind when using the tool

Please run the cell below before anything with ffmpeg. This is how you install ffmpeg on a binder notebook, thanks to [stack overflow](https://stackoverflow.com/questions/72217039/ffmpeg-and-jupyter-notebooks)

In [ ]:
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
print('')
!which ffmpeg
print('Done!')

Ok, we're ready to get started. Enter the name of the file you wish to use below. I recommend using one of the files in the notebook_scripts folder. I will not being using those API keys so they are less likely to have hit their 5-clips-a-day limit. You can also use any file with your own API key if you have one.

In [ ]:
filepath = "FILL IN FILE NAME" # "./notebook_scripts/notebook_script1.md"

In [ ]:
! rm -rf *.mp4 *.jpg *.ass
from parse import parse_from_file
from upload import upload_script, parse_upload_response, get_slides, get_avatar_clips
from compose import compose_scenes
import sys
import urllib.request
from transition import transitions
import ffmpeg
import time

script = parse_from_file(filepath)
if script:
    responses = upload_script(script)

    # parse the response content into the scenes - literally just the avatar video ids
    script = parse_upload_response(responses, script)

    # get the slides 
    script = get_slides(script, "./assets/")

    # then go get the links from the videos and download the clips. hopefully they've rendered by now
    #time.sleep(1500)
    script = get_avatar_clips(script, "./assets/")
    print(script)

    # compose the scenes
    script = compose_scenes(script)
    # transitions
    (script, v, a, v_d, a_d) = transitions(script)
    # output video
    ffmpeg.output(v,a, script[0]["Lecture Name"]+".mp4", pix_fmt='yuv420p').run()
    
    # presumably response has the URL of the pending video. for each of the clips get the url. for each one, download it.
    # can't do this section without higher API limit yet
    #print(responses)
else:
    print(script)